In [1]:
#Import Statements

#basic
import pandas as pd
import numpy as np
from math import log,exp

#visualization
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn

% pylab inline

#sklearn
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error, confusion_matrix,accuracy_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor

#other stats
from scipy import stats
from patsy import dmatrices

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


## Pre-processing

In [2]:
def preprocess(df):
    
    # Feature engineering for categorical variables captured as numbers
    # Turn MSSubClass into factors
    code = ['20', '30', '40', '45', '50', '60', '70', '75', '80', '85', '90', '120', '150', '160', '180', '190']
    strings = ['1-STORY 1946 & NEWER ALL STYLES', '1-STORY 1945 & OLDER', '1-STORY W/FINISHED ATTIC ALL AGES', '1-1/2 STORY - UNFINISHED ALL AGES',\
     '1-1/2 STORY FINISHED ALL AGES', '2-STORY 1946 & NEWER', '2-STORY 1945 & OLDER', '2-1/2 STORY ALL AGES', \
     'SPLIT OR MULTI-LEVEL', 'SPLIT FOYER', 'DUPLEX - ALL STYLES AND AGES', '1-STORY PUD (Planned Unit Development) - 1946 & NEWER',\
     '1-1/2 STORY PUD - ALL AGES', '2-STORY PUD - 1946 & NEWER', 'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER', \
     '2 FAMILY CONVERSION - ALL STYLES AND AGES']

    MSSubClass=dict(zip(code,strings))
    df['MSSubClass']=[MSSubClass[str(val)] for val in df['MSSubClass']]

    # Turn month sold into factors
    months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']
    df['MoSold'] = df['MoSold'].replace(to_replace = df['MoSold'].value_counts().index.sort_values(),value=months)
    
    # Feature engineering for age-related variables
    # Convert yearsold vs yearbuilt into age of house
    df['Age'] =  df['YrSold'] - df['YearBuilt']
    df = df.drop('YearBuilt', 1)
    # Convert yearsold vs. yearreomdadd into age of remodel. Adding 2 to eliminate any negative or 0 values
    df['AgeRem'] = (df['YrSold'] - df['YearRemodAdd'])+2
    df = df.drop('YearRemodAdd', 1)
    # Remove age of garage - many missing values (if no garage)
    #  no additional valuable information (garage captured in other variables); age of house more important for age
    df = df.drop('GarageYrBlt', 1)
   
    # Fill select variables with most common / mode where logical
    # Most masonry veneer type is None and area is 0
    df['MasVnrType'] = df['MasVnrType'].fillna('None')
    df['MasVnrArea'] = df['MasVnrArea'].fillna(0.0)
    # Most electrical is 'SBrkr'
    df['Electrical'] = df['Electrical'].fillna('SBrkr')
    df['LotFrontage'] = df['LotFrontage'].fillna(mean(df['LotFrontage']))
    
    return df

In [3]:
def get_dummies(X_df):
    # Dummify X data
    X_dummy = pd.get_dummies(X_df,dummy_na=True)
    return X_dummy

In [4]:
def level_cat(df_train,df_comp):
    traincols = list(df_train.columns.values)
    testcols = list(df_comp.columns.values)
    
    # Align train data columns to competition data columns
    missingcols1 = list(set(testcols)-set(traincols))
    for col in missingcols1:
        df_train[col] = 0.0

    # Align test data columns to competition data columns
    missingcols = list(set(traincols)-set(testcols))
    print len(missingcols)
    for col in missingcols:
        df_comp[col] = 0.0
    df_comp = df_comp[traincols+missingcols1]
    
    return df_train,df_comp

In [6]:
housedata = preprocess(pd.read_csv('train.csv'))
# Split into X and y
y = housedata['SalePrice']
X = housedata.drop('SalePrice', 1)

#preprocess and dummify competition
X = get_dummies(X)
competition = preprocess(pd.read_csv('test.csv'))
X_comp = get_dummies(competition)

X,X_comp = level_cat(X,X_comp)

IOError: File train.csv does not exist

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33, random_state=1)

## Initial Model Exploration
Work that was done in APM HW 3

In [ ]:
# Run models
#Linear Regression
MLR =LinearRegression()
model_MLR = MLR.fit(X_train,y_train)
MLR_predict = model_MLR.predict(X_test)
MLR_RMSE= sqrt(mean_squared_error(y_test, MLR_predict))
print "MLR RMSE:",MLR_RMSE

#get best λ with LassoCV
lasso_cv = LassoCV(alphas=None, cv=10, max_iter=10000)
model_cv = lasso_cv.fit(X_train,ravel(y_train))
print "Lasso CV best λ:",model_cv.alpha_
lasso_predict= model_cv.predict(X_test)
lasso_RMSE= sqrt(mean_squared_error(y_test, lasso_predict))
print "Lasso RMSE:",lasso_RMSE

#get best λ with RidgeCV
ridge_cv = RidgeCV(cv=10)
ridge_model_cv = ridge_cv.fit(X_train,ravel(y_train))
print "Ridge CV best λ:",ridge_model_cv.alpha_
ridge_predict= ridge_model_cv.predict(X_test)
ridge_RMSE= sqrt(mean_squared_error(y_test, ridge_predict))
print "Ridge RMSE:",ridge_RMSE

In [ ]:
#Model residuals

MLRresiduals = y_test - MLR_predict
lassoResiduals = y_test - lasso_predict
ridgeResiduals = y_test - ridge_predict

# Plot residuals
fig = plt.figure(figsize=(15, 7))

# Predicted vs. Actual
ax1 = fig.add_subplot(121)
ax1.plot(y_test,MLR_predict,"o",label='MLR', color='darkblue')
ax1.plot(y_test,lasso_predict,"o",label='Lasso',color='blue')
ax1.plot(y_test,ridge_predict,"o",label='Ridge',color='aqua')
ax1.legend(numpoints=1,loc='upper left')
ax1.set_ylabel('Predicted Price ($)')
ax1.yaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax1.set_xlabel('Actual Price ($)')
ax1.xaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax1.set_title('Predicted vs. Actual Price', fontsize=12, fontweight='bold')

# Residuals
ax2 = fig.add_subplot(122)
ax2.plot(y_test,MLRresiduals,"o",label='MLR',color='darkblue')
ax2.plot(y_test,lassoResiduals,"o",label='Lasso',color='blue')
ax2.plot(y_test,ridgeResiduals,"o",label='Ridge',color='aqua')
ax2.legend(numpoints=1,loc='upper left')
ax2.set_ylabel('Residuals ($)')
ax2.yaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax2.set_xlabel('Actual Price ($)')
ax2.xaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax2.set_title('Residuals', fontsize=12, fontweight='bold')

fig.subplots_adjust(wspace=.3)
plt.show()

In [ ]:
# Try predicting using log SalePrice
log_y_train = y_train.map(float).map(log)
log_y_test = y_test.map(float).map(log)

#Linear Regression
MLR =LinearRegression()
model_MLR2 = MLR.fit(X_train,log_y_train)
MLR_predict2 = model_MLR2.predict(X_test)
MLR_RMSE2 = sqrt(mean_squared_error(y_test, np.exp(MLR_predict2)))
print "MLR RMSE:",MLR_RMSE2

#get best λ with LassoCV
lasso_cv2 = LassoCV(alphas=None, cv=10, max_iter=10000)
model_cv2 = lasso_cv2.fit(X_train,ravel(log_y_train))
print "Lasso CV best λ:",model_cv2.alpha_
lasso_predict2= model_cv2.predict(X_test)
lasso_RMSE2 = sqrt(mean_squared_error(y_test, np.exp(lasso_predict2)))
print "Lasso RMSE:",lasso_RMSE2

#get best λ with RidgeCV
ridge_cv2 = RidgeCV(cv=10)
ridge_model_cv2 = ridge_cv2.fit(X_train,ravel(log_y_train))
print "Ridge CV best λ:",ridge_model_cv2.alpha_
ridge_predict2 = ridge_model_cv2.predict(X_test)
ridge_RMSE2 = sqrt(mean_squared_error(y_test, np.exp(ridge_predict2)))
print "Ridge RMSE:",ridge_RMSE2

In [ ]:
#residuals with log price models (in original scale)
MLRresiduals2 = y_test - np.exp(MLR_predict2)
lassoResiduals2 = y_test - np.exp(lasso_predict2)
ridgeResiduals2 = y_test - np.exp(ridge_predict2)

# Plot residuals
fig = plt.figure(figsize=(15, 7))

# Predicted vs. Actual
ax1 = fig.add_subplot(121)
ax1.plot(y_test,np.exp(MLR_predict2),"o",label='MLR',color='darkblue')
ax1.plot(y_test,np.exp(lasso_predict2),"o",label='Lasso',color='blue')
ax1.plot(y_test,np.exp(ridge_predict2),"o",label='Ridge',color='aqua')
ax1.legend(numpoints=1,loc='upper left')
ax1.set_ylabel('Predicted Price ($)')
ax1.yaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax1.set_xlabel('Actual Price ($)')
ax1.xaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax1.set_title('Predicted vs. Actual Price', fontsize=12, fontweight='bold')

# Residuals
ax2 = fig.add_subplot(122)
ax2.plot(y_test,MLRresiduals2,"o",label='MLR',color='darkblue')
ax2.plot(y_test,lassoResiduals2,"o",label='Lasso',color='blue')
ax2.plot(y_test,ridgeResiduals2,"o",label='Ridge',color='aqua')
ax2.legend(numpoints=1,loc='upper left')
ax2.set_ylabel('Residuals ($)')
ax2.yaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax2.set_xlabel('Actual Price ($)')
ax2.xaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax2.set_title('Residuals', fontsize=12, fontweight='bold')

fig.subplots_adjust(wspace=.3)
plt.show()

In [ ]:
# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_scaler = StandardScaler()
Y_scaled = ravel(y_scaler.fit_transform(y_train.reshape(-1, 1)))
Y_test_scaled = ravel(y_scaler.transform(y_test.reshape(-1, 1)))

In [ ]:
# Run MLP using different hidden layer sizes
sizes = [(20,),(50,),(100,),(150,),(200,),(250,),(300,)]
RMSEs = []
for size in sizes:
    MLP = MLPRegressor(hidden_layer_sizes = size,activation='tanh',solver='sgd',learning_rate='constant',random_state=42,\
                       batch_size=40,learning_rate_init=0.001)
    model_MLP = MLP.fit(X_scaled,Y_scaled)
    MLP_predict = model_MLP.predict(X_test_scaled)
    MLP_RMSE= sqrt(mean_squared_error(y_test, y_scaler.inverse_transform(MLP_predict)))
    RMSEs.append(MLP_RMSE)

In [ ]:
# Choosing hidden layer size with best RMSE
MLP = MLPRegressor(hidden_layer_sizes = sizes[np.argmin(RMSEs)],activation='tanh',solver='sgd',learning_rate='constant',random_state=42,\
                   batch_size=40,learning_rate_init=0.001)
model_MLP = MLP.fit(X_scaled,Y_scaled)
MLP_predict = model_MLP.predict(X_test_scaled)
MLP_RMSE = sqrt(mean_squared_error(y_test, y_scaler.inverse_transform(MLP_predict)))
print "MLP RMSE:",MLP_RMSE

MLPresiduals = y_test - y_scaler.inverse_transform(MLP_predict)

In [ ]:
# Plot residuals
fig = plt.figure(figsize=(15, 7))

# Predicted vs. Actual
ax1 = fig.add_subplot(121)
ax1.plot(y_test,y_scaler.inverse_transform(MLP_predict),"o",label='MLP',color='aliceblue')
ax1.legend(numpoints=1,loc='upper left')
ax1.set_ylabel('Predicted Price ($)')
ax1.yaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax1.set_xlabel('Actual Price ($)')
ax1.xaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax1.set_title('Predicted vs. Actual Price', fontsize=12, fontweight='bold')

# Residuals
ax2 = fig.add_subplot(122)
ax2.plot(y_test,MLPresiduals, "o",label='MLP',color='aliceblue')
ax2.legend(numpoints=1,loc='upper left')
ax2.set_ylabel('Residuals ($)')
ax2.yaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax2.set_xlabel('Actual Price ($)')
ax2.xaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax2.set_title('Residuals', fontsize=12, fontweight='bold')

fig.subplots_adjust(wspace=.3)
plt.show()

In [ ]:
# Try Linear Regression, Lasso, and Ridge with scaled X data

#Linear Regression
MLR =LinearRegression(fit_intercept=False)
model_MLR1 = MLR.fit(X_scaled,y_train)
MLR_predict1 = model_MLR1.predict(X_test_scaled)
MLR_RMSE1= sqrt(mean_squared_error(y_test, MLR_predict1))
print "MLR RMSE:",MLR_RMSE1

#get best λ with LassoCV
lasso_cv1 = LassoCV(alphas=None, cv=10, max_iter=10000)
model_cv1 = lasso_cv1.fit(X_scaled,ravel(y_train))
print "Lasso CV best λ:",model_cv1.alpha_
lasso_predict1= model_cv1.predict(X_test_scaled)
lasso_RMSE1= sqrt(mean_squared_error(y_test, lasso_predict1))
print "Lasso RMSE:",lasso_RMSE1

#get best λ with RidgeCV
ridge_cv1 = RidgeCV(cv=10)
ridge_model_cv1 = ridge_cv1.fit(X_scaled,ravel(y_train))
print "Ridge CV best λ:",ridge_model_cv1.alpha_
ridge_predict1= ridge_model_cv1.predict(X_test_scaled)
ridge_RMSE1= sqrt(mean_squared_error(y_test, ridge_predict1))
print "Ridge RMSE:",ridge_RMSE1

In [ ]:
# Based on RMSE, it appears as though MLR may have challenges predicting this data
# For example, the linear regression problem could be under-determined 
#  (where the number of linearly independent rows of the training matrix 
#    is less than its number of linearly independent columns),
#  resulting in erroneous predictions

# Graph to see
MLRresiduals1 = y_test - MLR_predict1
fig = plt.figure(figsize=(15, 7))

# Predicted vs. Actual
ax1 = fig.add_subplot(121)
ax1.plot(y_test,MLR_predict1,"o",label='MLR',color='darkblue')
ax1.legend(numpoints=1,loc='upper left')
ax1.set_ylabel('Predicted Price ($)')
ax1.set_xlabel('Actual Price ($)')
ax1.xaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax1.set_title('Predicted vs. Actual Price', fontsize=12, fontweight='bold')

# Residuals
ax2 = fig.add_subplot(122)
ax2.plot(y_test,MLRresiduals1, "o",label='MLR',color='darkblue')
ax2.legend(numpoints=1,loc='upper left')
ax2.set_ylabel('Residuals ($)')
ax2.set_xlabel('Actual Price ($)')
ax2.xaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax2.set_title('Residuals', fontsize=12, fontweight='bold')

fig.subplots_adjust(wspace=.3)
plt.show()

In [ ]:
# Plot Lasso and Ridge
lassoResiduals1 = y_test - lasso_predict1
ridgeResiduals1 = y_test - ridge_predict1

fig = plt.figure(figsize=(15, 7))

# Predicted vs. Actual
ax1 = fig.add_subplot(121)
ax1.plot(y_test,lasso_predict1,"o",label='Lasso',color='blue')
ax1.plot(y_test,ridge_predict1,"o",label='Ridge',color='aqua')
ax1.legend(numpoints=1,loc='upper left')
ax1.set_ylabel('Predicted Price ($)')
ax1.yaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax1.set_xlabel('Actual Price ($)')
ax1.xaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax1.set_title('Predicted vs. Actual Price', fontsize=12, fontweight='bold')

# Residuals
ax2 = fig.add_subplot(122)
ax2.plot(y_test,lassoResiduals1,"o",label='Lasso',color='blue')
ax2.plot(y_test,ridgeResiduals1,"o",label='Ridge',color='aqua')
ax2.legend(numpoints=1,loc='upper left')
ax2.set_ylabel('Residuals ($)')
ax2.yaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax2.set_xlabel('Actual Price ($)')
ax2.xaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax2.set_title('Residuals', fontsize=12, fontweight='bold')

fig.subplots_adjust(wspace=.3)
plt.show()

In [ ]:
# Try fitting on scaled data and log SalePrice for Lasso and Ridge
#get best λ with LassoCV
lasso_cv3 = LassoCV(alphas=None, cv=10, max_iter=10000)
model_cv3 = lasso_cv3.fit(X_scaled,ravel(log_y_train))
print "Lasso CV best λ:",model_cv3.alpha_
lasso_predict3= model_cv3.predict(X_test_scaled)
lasso_RMSE3 = sqrt(mean_squared_error(y_test, np.exp(lasso_predict3)))
print "Lasso RMSE:",lasso_RMSE3

#get best λ with RidgeCV
ridge_cv3 = RidgeCV(cv=10)
ridge_model_cv3 = ridge_cv3.fit(X_scaled,ravel(log_y_train))
print "Ridge CV best λ:",ridge_model_cv3.alpha_
ridge_predict3= ridge_model_cv3.predict(X_test_scaled)
ridge_RMSE3 = sqrt(mean_squared_error(y_test, np.exp(ridge_predict3)))
print "Ridge RMSE:",ridge_RMSE3

In [ ]:
# Plot Lasso and Ridge results for scaled X data / log SalePrice
lassoResiduals3 = y_test - np.exp(lasso_predict3)
ridgeResiduals3 = y_test - np.exp(ridge_predict3)

fig = plt.figure(figsize=(15, 7))

# Predicted vs. Actual
ax1 = fig.add_subplot(121)
ax1.plot(y_test,np.exp(lasso_predict3),"o",label='Lasso',color='blue')
ax1.plot(y_test,np.exp(ridge_predict3),"o",label='Ridge',color='aqua')
ax1.legend(numpoints=1,loc='upper left')
ax1.set_ylabel('Predicted Price ($)')
ax1.yaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax1.set_xlabel('Actual Price ($)')
ax1.xaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax1.set_title('Predicted vs. Actual Price', fontsize=12, fontweight='bold')

# Residuals
ax2 = fig.add_subplot(122)
ax2.plot(y_test,lassoResiduals3,"o",label='Lasso',color='blue')
ax2.plot(y_test,ridgeResiduals3,"o",label='Ridge',color='aqua')
ax2.legend(numpoints=1,loc='upper left')
ax2.set_ylabel('Residuals ($)')
ax2.yaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax2.set_xlabel('Actual Price ($)')
ax2.xaxis.set_major_formatter(mtick.FuncFormatter('{:,.0f}'.format))
ax2.set_title('Residuals', fontsize=12, fontweight='bold')

fig.subplots_adjust(wspace=.3)
plt.show()

### SVR

In [ ]:
best = SVR(kernel='linear', C=10)
best.fit(X_train,y_train)
predictions=best.predict(X_test)
np.sqrt(mean_squared_error(y_test,predictions))

### SGD Regressor

In [ ]:
parameters = {'loss': ['squared_loss', 'huber'],
             'penalty': ['l1','l2'],
              
             }
accuracy=make_scorer(mean_squared_error, greater_is_better=False)
clf2 = GridSearchCV(SGDRegressor(random_state=42), parameters, cv=5,scoring=accuracy)
clf2.fit(X_train, y_train)

In [ ]:
print clf2.best_params_
print clf2.best_score_

In [ ]:
best2 = SGDRegressor(loss='huber', penalty='l2',random_state=42)
best2.fit(X_train, y_train)
pred = best2.predict(X_test)
np.sqrt(mean_squared_error(y_test, pred))